# Load in packages

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import winsound
import os
import pickle

# Get the weblinks

In [4]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'}


In [182]:
# Scrape the weblinks per letter, with a variable number of pages
productlinks_Z = []
letter = "Z"
pages = 1

for page in range(1,pages+1):
    new_url = f"https://www.whisky.com/whisky-database/bottle-search.html?tx_datamintsflaschendb_pi4%5Baction%5D=index&tx_datamintsflaschendb_pi4%5Bcontroller%5D=Flaschen&tx_datamintsflaschendb_pi4%5BcurPage%5D={page}&tx_datamintsflaschendb_pi4%5BsearchCriteria%5D%5BtitleStartsWith%5D={letter}"
    response = requests.get(new_url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Check if there are more pages to scrape
    productlist = soup.find_all('div', class_='title')
    
    for item in productlist:
        for link in item.find_all('a', href=True):
            productlinks_Z.append('https://www.whisky.com/'+link['href'])

    page += 1

duration = 1000  # milliseconds
freq = 440  # Hz
winsound.Beep(freq, duration)

In [185]:
productlinks = productlinks + productlinks_Z

In [186]:
len(productlinks)

80777

#### Remove duplicates from links

In [381]:
whisky_all_links = list(set(productlinks))

In [382]:
len(whisky_all_links)

36740

#### Pickle the productlinks list

In [3]:
import pickle

In [383]:
with open("whisky_all_links", "wb") as fp:
    pickle.dump(whisky_all_links, fp)

In [4]:
with open("whisky_all_links", "rb") as fp:
    whisky_all_links = pickle.load(fp)

In [5]:
len(whisky_all_links)

36740

# Create lists of random weblinks (no replacement)

In [7]:
import random
random.seed(333)

# create the original list
original_list = whisky_all_links

# create a list to store the 55 separate lists
whisky_separate_lists_of_links = []

# iterate to create the 55 separate lists of 167x4 items
for i in range(54):
    # create a copy of the original list
    copy_list = original_list.copy()

    # create a new list for the current iteration
    new_list = []

    # randomly select 167 unique items from the copy list
    while len(new_list) < 668:
        index = random.randrange(len(copy_list))
        selected_item = copy_list.pop(index)

        # check if the selected item is already present in the new list
        is_duplicate = False
        for existing_item in new_list:
            if selected_item == existing_item:
                is_duplicate = True
                break

        # if the selected item is not a duplicate, add it to the new list
        if not is_duplicate:
            new_list.append(selected_item)

    # append the new list to the list of separate lists
    whisky_separate_lists_of_links.append(new_list)

In [ ]:
for i in whisky_separate_lists_of_links:
    print(len(i))

In [9]:
with open("whisky_separate_lists_of_links", "wb") as fp:
    pickle.dump(whisky_separate_lists_of_links, fp)

In [5]:
with open("whisky_separate_lists_of_links", "rb") as fp:
    whisky_separate_lists_of_links = pickle.load(fp)

# Get information from webpage

In [ ]:
product_title = []
distillery = []
country = []
region = []
age = []
abv = []
bottle_size = []
bottler = []
maturing = []
nbr_of_bottles = []
treatment = []
rating = []
nbr_of_ratings = []
nosing_types = []
nosing_ratings = []
tasting_types = []
tasting_ratings = []
finish_types = []
finish_ratings = []

list_number = 54

active_list = whisky_separate_lists_of_links[list_number]

for i in range(len(active_list)):
    r = requests.get(active_list[i], headers=headers)
    soup = BeautifulSoup(r.content, 'html.parser')
    
    #PRODUCT TITLE
    try:
        product_title.append(soup.find('meta', {'property': 'og:title'}).get('content').strip())
    except:
        product_title.append("N/A")
    
    #DISTILLERY
    try:
        distilleries = []
        for i in soup.find_all('a', href=True):
            if '/whisky-database/distilleries/details/' in i.get('href'):
                distilleries.append(i.get_text())
        distillery.append(distilleries[0])
    except:
        distillery.append("N/A")
    
    #COUNTRY
    try:
        counter = 0
        for i in soup.find_all('a', href=True):
            if '5Bland' in i.get('href'):
                country.append(i.get_text())
                counter += 1
        if counter == 0:
            country.append("N/A")
    except:
        country.append("N/A")
    
    #REGION
    try:
        counter = 0
        for i in soup.find_all('a', href=True):
            if '5Bregion' in i.get('href'):
                region.append(i.get_text())
                counter += 1
        if counter == 0:
            region.append("N/A")
    except:
        region.append("N/A")

    #AGE
    try:
        age.append(soup.find('tr', class_='fassnummern').find('span',class_='value').get_text().strip())
    except:
        age.append("N/A")
    
    #ABV
    try:
        abv.append(soup.find('tr', class_='alkoholgehalt').find('span',class_='value').get_text().strip())
    except:
        abv.append("N/A")
    
    #Bottle_Size
    try:
        bottle_size.append(soup.find('tr', class_='flaschengroesse').find('span',class_='value').get_text().strip())
    except:
        bottle_size.append("N/A")
    
    #Bottler
    try:
        bottlers = []
        for i in soup.find_all('a', href=True):
            if '/whisky-database/bottler/' in i.get('href'):
                bottlers.append(i.get_text())
        bottler.append(bottlers[0])
    except:
        bottler.append("N/A")
    
    #Maturing
    try:
        maturing_string = soup.find('tr', class_='casktypes maturing-level-1').find('span',class_='value').get_text().strip()
        maturing_final_string = re.sub(r"\s+", "", maturing_string)
        maturing.append(maturing_final_string)
    except:
        maturing.append("N/A")
    
    #Nbr_of_bottles
    try:
        nbr_of_bottles.append(soup.find('tr', class_='anzahlFlaschen').find('span',class_='value').get_text().strip())
    except:
        nbr_of_bottles.append("N/A")
        
    #Treatment
    try:
        img_tags = soup.find('tr', class_='merkmale').find('span',class_='value').find_all('img')
        treatment.append([img_tag.get('alt') for img_tag in img_tags])
    except:
        treatment.append("N/A")
    
    #Rating
    try:
        div_rating = soup.find_all('div', class_='ratings statistics')
        rating.append(div_rating[0].get_text().strip()[:4].strip())
    except:
        rating.append("N/A")
    
    #Nbr_of_ratings
    try:
        nbr_of_ratings.append(soup.find('div', class_='rating-summary').get_text(strip=True))
    except:
        nbr_of_ratings.append("N/A")

    #FINISH
    try:
        div_finish = soup.find_all('div', {'data-limit-children-more-link-container-selector':'#tx-datamintsflaschendb-tasteiconstatistics-finish-more-container'})
        finish_data = div_finish[0].find_all('div', class_="item tasteicon-statistic")

        indiv_finish = []
        indiv_finish_ratings = []

        for i in finish_data:
            indiv_finish.append(i.find('div',class_='title left').get_text().strip())
            rating_comments = i.find_all(string=lambda text: 'Rating' in str(text))
            indiv_finish_ratings.append([comment.split(':')[-1].strip() for comment in rating_comments])
        
        finish_types.append(indiv_finish)
        finish_ratings.append(indiv_finish_ratings)

    except:
        finish_types.append('N/A')
        finish_ratings.append('N/A')

    #NOSINGS
    try:
        div_nosing = soup.find_all('div', {'data-limit-children-more-link-container-selector':'#tx-datamintsflaschendb-tasteiconstatistics-nosing-more-container'})
        nosing_data = div_nosing[0].find_all('div', class_="item tasteicon-statistic")

        indiv_nosing = []
        indiv_nosing_ratings = []

        for i in nosing_data:
            indiv_nosing.append(i.find('div',class_='title left').get_text().strip())
            rating_comments = i.find_all(string=lambda text: 'Rating' in str(text))
            indiv_nosing_ratings.append([comment.split(':')[-1].strip() for comment in rating_comments])
        
        nosing_types.append(indiv_nosing)
        nosing_ratings.append(indiv_nosing_ratings)

    except:
        nosing_types.append('N/A')
        nosing_ratings.append('N/A')

    #TASTINGS
    try:
        div_tasting = soup.find_all('div', {'data-limit-children-more-link-container-selector':'#tx-datamintsflaschendb-tasteiconstatistics-tasting-more-container'})
        tasting_data = div_tasting[0].find_all('div', class_="item tasteicon-statistic")

        indiv_tasting = []
        indiv_tasting_ratings = []

        for i in tasting_data:
            indiv_tasting.append(i.find('div',class_='title left').get_text().strip())
            rating_comments = i.find_all(string=lambda text: 'Rating' in str(text))
            indiv_tasting_ratings.append([comment.split(':')[-1].strip() for comment in rating_comments])
        
        tasting_types.append(indiv_tasting)
        tasting_ratings.append(indiv_tasting_ratings)

    except:
        tasting_types.append('N/A')
        tasting_ratings.append('N/A')

dataset = {
    'product_title':product_title,
    'distillery':distillery,
    'country':country,
    'region':region,
    'age':age,
    'abv':abv,
    'bottle_size':bottle_size,
    'bottler':bottler,
    'maturing':maturing,
    'nbr_of_bottles':nbr_of_bottles,
    'treatment':treatment,
    'rating':rating,
    'nbr_of_ratings':nbr_of_ratings,
    'finish_types':finish_types,
    'finish_ratings':finish_ratings,
    'nosing_types':nosing_types,
    'nosing_ratings':nosing_ratings,
    'tasting_types':tasting_types,
    'tasting_ratings':tasting_ratings,
}

pd.DataFrame(dataset).to_csv(f'df_{list_number}.csv')
print(f"df_{list_number} = DONE")

duration = 1000  # milliseconds
freq = 440  # Hz
winsound.Beep(freq, duration)

#### Code to test individual scrapings (DO NOT RUN IF NOT NEEDED)

In [ ]:
## TEST CELL
country = []
region = []

for i in range(1,167):
    r = requests.get(active_list[i], headers=headers)
    soup = BeautifulSoup(r.content, 'html.parser')

    countries = []
    for k in soup.find_all('a', href=True):
        if '5Bland' in k.get('href'):
            country = k.get_text()
    print(f"{i} - {country}")

In [352]:
active_list[13]

'https://www.whisky.com//whisky-database/details/dalwhinnie.html'

In [365]:
r = requests.get('https://www.whisky.com//whisky-database/details/jim-beam-rye-2.html', headers=headers)
soup = BeautifulSoup(r.content, 'html.parser')

soup_1 = soup.find_all('a', href=True)
soup_2 = soup_1.find_all()

123

In [348]:
r = requests.get('https://www.whisky.com//whisky-database/details/jim-beam-rye-2.html', headers=headers)
soup = BeautifulSoup(r.content, 'html.parser')

countries = []
for k in soup.find_all('a', href=True):
    if 'land' in k.get('href'):
        countries.append(k.get_text())
country.append(countries[0])
print(f"{i} - {countries[0]}")

21 - USA


In [ ]:
## TEST CELL
country = []
region = []

for i in range(1,167):
    r = requests.get(active_list[i], headers=headers)
    soup = BeautifulSoup(r.content, 'html.parser')

    countries = []
    for k in soup.find_all('a', href=True):
        if 'land' in k.get('href'):
            countries.append(k.get_text())
    print(f"{i} - {countries[0]}")

In [371]:
country = []
region = []
product = []

for i in range(len(active_list)):
    r = requests.get(active_list[i], headers=headers)
    soup = BeautifulSoup(r.content, 'html.parser')
    
    product.append(active_list[i])
    
    #COUNTRY
    try:
        counter = 0
        for i in soup.find_all('a', href=True):
            if '5Bland' in i.get('href'):
                country.append(i.get_text())
                counter += 1
        if counter == 0:
            country.append("N/A")

    except:
        country.append("N/A")
    

In [372]:
print(f"len product = {len(product)}")
print(f"len country = {len(country)}")

len product = 167
len country = 167


# Join all files

In [2]:
# Specify the path to your folder containing CSV files
folder_path = 'C:/Users/Dieter Carels/OneDrive - Minds Alike/LEARNING/POSTGRADUAAT BIG DATA/DISSERTATION/DATA'

# Get a list of all CSV files in the folder
csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

# Initialize an empty DataFrame to store the combined data
final_raw_data = pd.DataFrame()

# Iterate through each CSV file and concatenate its data to the combined DataFrame
for file in csv_files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)
    final_raw_data = pd.concat([final_raw_data, df], ignore_index=True)

# Display the combined DataFrame
final_raw_data


,Unnamed: 0,product_title,distillery,country,region,age,abv,bottle_size,bottler,maturing,nbr_of_bottles,treatment,rating,nbr_of_ratings,finish_types,finish_ratings,nosing_types,nosing_ratings,tasting_types,tasting_ratings
0,0,Talisker 10 Years,Talisker,Scotland,Islands,10 Years,45.8%,0.7 l,Original bottling,NaN,327 Bottles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Glen Moray Elgin Classic,Glen Moray,Scotland,Speyside,NaN,40%,0.7 l,Original bottling,NaN,228 Bottles,"['Chill filtration - With', 'Artifical colouri...",3.4,Number of Ratings:24,"['Spices:', 'Sweet:', 'Sweet:', 'Malt:', 'Malt...","[['1.9565', '1.8156'], ['1.9565', '1.9104'], [...","['Sweet:', 'Fruit:', 'Fruit:', 'Green Apple:',...","[['3.2609', '2.3018'], ['2.8261', '2.0004'], [...","['Fruit:', 'Sweet:', 'Fruit:', 'Sweet:', 'Pear...","[['3.4783', '2.338'], ['3.2609', '2.1587'], ['..."
2,2,Remedy Spiced Rum + Elixir &amp; Pineapple Min...,NaN,NaN,NaN,NaN,41%,0.8 l,Original bottling,NaN,NaN,"['Chill filtration - With', 'Artifical colouri...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,3 Years Stockwerk,Emil Scheibel,Germany,NaN,3 Years,46%,0.7 l,Original bottling,"Bourbon,DeutscheEiche,FranzösischeEiche",7 Bottles,"['Chill filtration - Without', 'Artifical colo...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Williamson 16 Years - 2005,Laphroaig,Scotland,Islay,16 Years,52.6%,0.7 l,A.D. Rattray,Bourbon,214 Bottles,"['Chill filtration - Without', 'Artifical colo...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35399,663,Famous Grouse,Blend-Scotch,Scotland,NaN,NaN,40%,1 l,Original bottling,"Bourbon,Sherry-Fino","1,329 Bottles","['Chill filtration - With', 'Artifical colouri...",2.8,Number of Ratings:97,"['Sweet:', 'Spices:', 'Oak:', 'Nuts:', 'Malt:'...","[['0.8333', '1.2341'], ['0.6061', '0.8531'], [...","['Zitrus:', 'Oak:', 'Fruit:', 'Sweet:', 'Malt:...","[['2.5', '3.899'], ['1.7424', '2.404'], ['1.21...","['Spices:', 'Peat Smoke:', 'Nuts:', 'Oil:', 'S...","[['2.3485', '2.5395'], ['2.2727', '2.2819'], [..."
35400,664,Finch 5 Years - 2008 Dinkel-Port 2013,Finch,Germany,NaN,5 Years,41%,0.5 l,NaN,Wein,51 Bottles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35401,665,Highland Park 40 Years,Highland Park,Scotland,Islands,40 Years,48.3%,0.05 l,Original bottling,NaN,568 Bottles,NaN,5.0,Number of Ratings:1,NaN,NaN,NaN,NaN,NaN,NaN
35402,666,NaN,unknown,NaN,NaN,NaN,45%,0.5 l,Original bottling,NaN,"3,115 Bottles",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
final_raw_data.to_csv('FINAL_RAW_DATA.csv',index=False)